In [1]:
import sqlite3
import pandas as pd

db_file = r'D:\recomments_product\recomment_product\db.sqlite3'

# Connect to the SQLite database
connection = sqlite3.connect(db_file)

# Create a cursor object
cursor = connection.cursor()
def fetch_table_as_df(cursor, table_name):
    query = f"SELECT * FROM {table_name}"
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]  
    df = pd.DataFrame(rows, columns=columns)
    df.drop(df.columns[0], axis=1, inplace=True)
    
    return df

# Fetch data
df_product = fetch_table_as_df(cursor, "shop_product")
df_user = fetch_table_as_df(cursor, "auth_user")
df_rating = fetch_table_as_df(cursor, "shop_rating")
df_search_history = fetch_table_as_df(cursor, "shop_searchhistory")
df_view_history = fetch_table_as_df(cursor, "shop_viewhistory")

# Close connection
connection.close()



# Search product by Image

1. preprocessing data

In [2]:

# Embedding data

# Data for task process
# df_product 
# df_user 
# df_search_history



##############################################################################################################
#############################################################################################################
# View data 
print("Product table!")
print(df_product.head())
print("User table!")
print(df_user.head())
print("Search history table!")
print(df_search_history.head())


Product table!
       image  price articleNumber articleType  \
0  10000.jpg  834.0    5410480144      Skirts   
1  10001.jpg  834.0    5410480144      Skirts   
2  10002.jpg  834.0    5410480144      Skirts   
3  10003.jpg  834.0    5410480144     Tshirts   
4  10004.jpg  834.0    5410480144      Shorts   

                             productDisplayName masterCategory subCategory  \
0      Palm Tree Girls Sp Jace Sko White Skirts        Apparel  Bottomwear   
1  Palm Tree Kids Girls Sp Jema Skt Blue Skirts        Apparel  Bottomwear   
2        Palm Tree Kids Sp Jema Skt Blue Skirts        Apparel  Bottomwear   
3        Nike Women As Nike Eleme White T-Shirt        Apparel     Topwear   
4             Nike Men As 7 Sw Temp Grey Shorts        Apparel  Bottomwear   

  gender baseColour fashionType  season    year    usag  
0  Women      White     Fashion  Summer  2011.0  Casual  
1  Women       Blue     Fashion  Summer  2011.0  Casual  
2  Women       Blue     Fashion  Summer  2011.0

# Check product table 


In [3]:
df_product.head()

articleType = df_product['articleType'].value_counts()
gender = df_product['gender'].value_counts()
baseColour = df_product['baseColour'].value_counts()
fashionType = df_product['fashionType'].value_counts()
season = df_product['season'].value_counts()


print("acticleType")
print(articleType)
print("gender")
print(gender)
print("baseColour")
print(baseColour)
print("fashionType")
print(fashionType)
print("season")
print(season)


acticleType
articleType
Tshirts                7067
Shirts                 3217
Casual Shoes           2845
Watches                2542
Sports Shoes           2036
                       ... 
Ipad                      1
Body Wash and Scrub       1
Mens Grooming Kit         1
Hair Accessory            1
Shoe Laces                1
Name: count, Length: 143, dtype: int64
gender
gender
Men       22147
Women     18631
Unisex     2161
Boys        830
Girls       655
Name: count, dtype: int64
baseColour
baseColour
Black                9728
White                5538
Blue                 4918
Brown                3494
Grey                 2741
Red                  2455
Green                2115
Pink                 1860
Navy Blue            1789
Purple               1640
Silver               1090
Yellow                778
Beige                 749
Gold                  628
Maroon                581
Orange                530
Olive                 410
Multi                 394
Cream              

# Classificaion model

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from PIL import Image
from torch.utils.data import random_split
import os

In [ ]:


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class CustomDataset(Dataset):
    def __init__(self, img_names, targets, img_dir, transform=None):
        self.img_paths = [os.path.join(img_dir, img_name) for img_name in img_names]
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, self.targets[idx]

dataset = CustomDataset(
    img_names=df_product['image'],
    targets=df_product['articleType'],
    img_dir=r"D:\recomments_product\recomment_product\media",
    transform=transform
)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


# Define dataset sizes
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Split the dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



278